## **Building an Intelligent Scanning Agent for Identifying Deals Using RSS Feeds**


### **Objectives:**
1. Integrating RSS (Really Simple Syndication) feeds in deal monitoring.
2. Scrape and filter deals using a ScanningAgent.
3. Implement a GPT-powered selection mechanism for filtering high-quality deals.
4. Develop an automated pipeline for deal selection and analysis.


### **Introduction to RSS Feeds**
RSS (Really Simple Syndication) feeds allow users to receive updates from various sources automatically. This technology is widely used for tracking news, blogs, and online deals. In this session, we will build a ScanningAgent that continuously scans RSS feeds for the best available deals.

### **1. Importing Required Libraries**


In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from Core.deals import ScrapedDeal, DealSelection

### **2. Setting Up the Environment**


In [ ]:
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

### **3. Scraping Deals from RSS Feeds**


In [ ]:
deals = ScrapedDeal.fetch(show_progress=True)
len(deals)

- The `fetch()` method retrieves a list of deals from predefined RSS feeds.
- The length of the `deals` list represents the number of deals scraped.

### **4. Describing Individual Deals**


In [ ]:
deals[44].describe()


### **5. Defining a System Prompt for Filtering Deals**


In [ ]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

- This prompt instructs the GPT model to filter out low-quality deals and summarize product descriptions concisely.
- The model ensures price clarity and avoids misleading discounts.

### **6. Formatting the User Request**


In [ ]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [ ]:
print(user_prompt[:2000])

- This creates a user input prompt containing all deal descriptions.
- It ensures that only deals with clear pricing are considered.

### **7. Generating GPT-Powered Deal Recommendations**


In [ ]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

- This function sends deal descriptions to the GPT model.
- The model selects the top five most detailed deals based on product descriptions.

### **8. Running the Recommendation Model**


In [ ]:
result = get_recommendations()
len(result.deals)

In [ ]:
result.deals[1]

- The function returns the filtered list of deals.
- We can inspect the selected deals.

### **9. Automating with a ScanningAgent**


In [ ]:
from agents.scanner_agent import ScannerAgent
agent = ScannerAgent()
result = agent.scan()
result

- The `ScannerAgent` automates the scanning process, ensuring continuous deal updates.
- The `scan()` method retrieves and filters deals automatically.